# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [69]:
import os, requests
import random
from dotenv import load_dotenv
from bs4 import BeautifulSoup

load_dotenv()
API_KEY = os.environ.get('key')
REGION = 'na1' 
root = 'https://americas.api.riotgames.com'
winRateDict = {}
pickRateDict = {}
top5WR = {}
champList = ['aphelios', 'ashe', 'caitlyn', 'corki', 'draven', 'ezreal', 'jhin', 'jinx', 'kaisa', 'kalista', 'kogmaw', 'lucian', 'missfortune', 'nilah', 'samira', 'sivir', 'smolder', 'tristana', 'twitch', 'varus', 'vayne', 'xayah', 'zeri']
champListLength = len(champList)
headers = {
    'User-Agent': 'Mozilla/5.0 (compatible; YourAppName/1.0)'
}

In [78]:
def updateWinRates(winRateDict, headers):
    for champ in champList:
        opggURL = f'https://www.op.gg/champions/{champ}/build'
        response = requests.get(opggURL, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            winRateEm = soup.find('em', string='Win rate')
            winRateB = winRateEm.find_next('b', class_='text-[12px]')
            if winRateB:
                raw_text = winRateB.get_text(separator=" ", strip=True)
                winRate = raw_text.replace('"', '').strip()
                winRateDict.update({champ: winRate.strip(" %")})
        else:
            print(f"Failed to retrieve data: {response.status_code}")
            
def updatePickRates(pickRateDict, headers):
    for champ in champList:
        opggURL = f'https://www.op.gg/champions/{champ}/build'
        response = requests.get(opggURL, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            pickRateEm = soup.find('em', string='Pick rate')
            pickRateB = pickRateEm.find_next('b', class_='text-[12px]')
            if pickRateB:
                raw_text = pickRateB.get_text(separator=" ", strip=True)
                pickRate = raw_text.replace('"', '').strip()
                pickRateDict.update({champ: pickRate.strip(" %")})
        else:
            print(f"Failed to retrieve data: {response.status_code}")

def updateTop5WR(top5WR, winRateDict):
    for key, value in winRateDict.items():
        currWR = float(winRateDict.get(key))
        if len(top5WR) < 5:
            top5WR[key] = currWR
        else:
            smallest_key = min(top5WR, key=top5WR.get)
            if currWR > top5WR[smallest_key]:
                top5WR.pop(smallest_key)
                top5WR[key] = currWR

def getRandomTop5WR(top5WR):
    keyList = list(top5WR.keys())
    randomKey = random.choice(keyList)
    print(f'Your random champion is {randomKey.capitalize()}!\nWin rate: {top5WR[randomKey]}')
    
def pickRandomChamp(champList):
    return champList[random.randint(0, champListLength - 1)]

In [71]:
updateWinRates(winRateDict, headers)

In [72]:
updatePickRates(pickRateDict, headers)

In [73]:
updateTop5WR(top5WR, winRateDict)

In [108]:
getRandomTop5WR(top5WR)

Your random champion is Kogmaw!
Win rate: 53.59
{'jinx': 52.01, 'kogmaw': 53.59, 'nilah': 53.85, 'tristana': 51.71, 'vayne': 52.14}
